In [1]:
import tensorflow as tf
tf.test.gpu_device_name()
from tensorflow.python.client import device_lib
device_lib.list_local_devices()
#!cat /proc/meminfo

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 15198730708779212438, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 11176148992
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 9268801139962906831
 physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"]

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [13]:
!ls "/content/drive/My Drive/Colab Notebooks/CV medical/data"

test_targets.npy   train_tensors.npy  weights.best.VGGmax.hdf5
test_tensors.npy   valid_targets.npy
train_targets.npy  valid_tensors.npy


In [0]:
import keras 
import numpy as np
from sklearn.datasets import load_files       
from keras.utils import np_utils
from glob import glob
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential
train_tensors=np.load('/content/drive/My Drive/Colab Notebooks/CV medical/data/train_tensors.npy')
valid_tensors=np.load('/content/drive/My Drive/Colab Notebooks/CV medical/data/valid_tensors.npy')
test_tensors=np.load('/content/drive/My Drive/Colab Notebooks/CV medical/data/test_tensors.npy')
train_targets=np.load('/content/drive/My Drive/Colab Notebooks/CV medical/data/train_targets.npy')
valid_targets=np.load('/content/drive/My Drive/Colab Notebooks/CV medical/data/valid_targets.npy')
test_targets=np.load('/content/drive/My Drive/Colab Notebooks/CV medical/data/test_targets.npy')

In [0]:
def add_new_last_layer(base_model, nb_classes):
    x = base_model.output
    x= GlobalAveragePooling2D()(x)
    #x=Dense(256,activation="relu")(x)
    #x=Dropout(0.3)(x)
    #x=Dense(256,activation="relu")(x)
    #x=Dropout(0.4)(x)
    predictions = Dense(nb_classes, activation='softmax')(x) 
    model = Model(inputs=base_model.input, outputs=predictions)
    return model
def setup_to_finetune(model):
    for layer in model.layers[:1]:
        layer.trainable = False
    for layer in model.layers[1:]:
        layer.trainable = True
    model.compile(optimizer=keras.optimizers.SGD(lr=0.001, momentum=0.9), loss='binary_crossentropy', metrics=['accuracy'])
from keras.applications.vgg19 import VGG19
from keras.models import Model
from keras.layers import Input, Dense
from keras import optimizers
base_model = VGG19(weights='imagenet', include_top=False) 
model = add_new_last_layer(base_model, 3)              
setup_to_finetune(model) 

In [20]:
from keras.callbacks import ModelCheckpoint  
checkpointer = ModelCheckpoint(filepath='weights.best.VGGmax.hdf5', 
                               verbose=1, save_best_only=True)
trainablevgg=model.fit(train_tensors, train_targets, 
                       validation_data=(valid_tensors, valid_targets),
                       epochs=20, batch_size=100, callbacks=[checkpointer], verbose=1)


Train on 2000 samples, validate on 150 samples
Epoch 1/20
2000/2000 [==============================] - 75s 37ms/step - loss: 0.0776 - acc: 0.9713 - val_loss: 0.5560 - val_acc: 0.8289

Epoch 00001: val_loss improved from inf to 0.55603, saving model to weights.best.VGGmax.hdf5
Epoch 2/20
2000/2000 [==============================] - 75s 37ms/step - loss: 0.0788 - acc: 0.9743 - val_loss: 0.7257 - val_acc: 0.7978

Epoch 00002: val_loss did not improve from 0.55603
Epoch 3/20
2000/2000 [==============================] - 75s 37ms/step - loss: 0.0963 - acc: 0.9652 - val_loss: 0.4780 - val_acc: 0.8022

Epoch 00003: val_loss improved from 0.55603 to 0.47796, saving model to weights.best.VGGmax.hdf5
Epoch 4/20
2000/2000 [==============================] - 75s 38ms/step - loss: 0.0434 - acc: 0.9890 - val_loss: 0.5484 - val_acc: 0.8356

Epoch 00004: val_loss did not improve from 0.47796
Epoch 5/20
2000/2000 [==============================] - 75s 37ms/step - loss: 0.0187 - acc: 0.9970 - val_loss: 0.

In [23]:
model_predictions = [np.argmax(model.predict(np.expand_dims(feature, axis=0))) for feature in test_tensors]
test_accuracy = 100*np.sum(np.array(model_predictions)==np.argmax(test_targets, axis=1))/len(model_predictions)
print('Test accuracy: %.4f%%' % test_accuracy)

Test accuracy: 72.0000%


In [22]:
print model_predictions

[1, 0, 1, 1, 1, 1, 1, 2, 1, 0, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 0, 1, 1, 1, 1, 0, 2, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 2, 1, 1, 0, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 0, 1, 0, 1, 0, 2, 2, 2, 1, 1, 0, 1, 1, 0, 0, 0, 2, 1, 1, 0, 2, 1, 2, 1, 1, 0, 2, 1, 0, 2, 1, 1, 1, 0, 1, 1, 2, 0, 1, 1, 0, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 1, 2, 2, 1, 1, 2, 1, 2, 1, 2, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 2, 1, 2, 1, 1, 1, 0, 1, 0, 2, 2, 0, 1, 2, 1, 1, 2, 2, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 0, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 2, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 2, 0, 1, 1, 1, 0, 1, 2, 2, 1, 2, 0, 1, 1, 1, 1, 1, 1, 2, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 0, 1, 1, 0, 1, 2, 1, 1, 1, 1, 1, 2, 1, 2, 1, 2, 1, 1, 2, 2, 1, 1, 1, 2, 1, 1, 1, 2, 0, 2, 0, 2, 1, 1, 0, 1, 1, 2, 1, 2, 1, 2, 0, 1, 0, 0, 2, 2, 2, 2, 1, 1, 1, 2, 1, 1, 

In [0]:
#!kill -9 -1